In [1]:
import os
import warnings
import copy
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data.examples import get_stallion_data
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

warnings.filterwarnings("ignore")

os.chdir("../../..")

/Users/zfwang/Applications/miniconda3/envs/3.10.9/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/zfwang/Applications/miniconda3/envs/3.10.9/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <2D89372E-C77C-3351-8455-B2E540126864> /Users/zfwang/Applications/miniconda3/envs/3.10.9/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <83256294-85FD-3E7B-8B5A-C1B09739EB70> /Users/zfwang/Applications/miniconda3/envs/3.10.9/lib/python3.10/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# load data
data = get_stallion_data()
print(data.head(), "\n")
print(data.shape, "\n")
print(data.columns)

        agency     sku    volume       date  industry_volume  soda_volume   
0    Agency_22  SKU_01   52.2720 2013-01-01        492612703    718394219  \
238  Agency_37  SKU_04    0.0000 2013-01-01        492612703    718394219   
237  Agency_59  SKU_03  812.9214 2013-01-01        492612703    718394219   
236  Agency_11  SKU_01  316.4400 2013-01-01        492612703    718394219   
235  Agency_05  SKU_05  420.9093 2013-01-01        492612703    718394219   

     avg_max_temp  price_regular  price_actual    discount  ...  labor_day   
0       25.845238    1168.903668   1069.166193   99.737475  ...          0  \
238     26.505000    1852.273642   1611.466298  240.807344  ...          0   
237     22.219737    1270.795012   1197.184260   73.610752  ...          0   
236     25.360000    1176.155397   1082.757488   93.397909  ...          0   
235     24.079012    1327.003396   1207.822992  119.180404  ...          0   

     independence_day  revolution_day_memorial  regional_games   
0 

In [3]:
# add time index
data["time_idx"] = data["date"].dt.year * 12 + data["date"].dt.month
data["time_idx"] -= data["time_idx"].min()

In [4]:
# add additional features
data["month"] = data.date.dt.month.astype(str).astype("category")  # categories have be strings
data["log_volume"] = np.log(data.volume + 1e-8)
data["avg_volume_by_sku"] = data.groupby(["time_idx", "sku"], observed=True).volume.transform("mean")
data["avg_volume_by_agency"] = data.groupby(["time_idx", "agency"], observed=True).volume.transform("mean")

In [5]:
# encode special days(one-hot)
special_days = [
    "easter_day",
    "good_friday",
    "new_year",
    "christmas",
    "labor_day",
    "independence_day",
    "revolution_day_memorial",
    "regional_games",
    "fifa_u_17_world_cup",
    "football_gold_cup",
    "beer_capital",
    "music_fest",
]
data[special_days] = data[special_days].apply(lambda x: x.map({0: "-", 1: x.name})).astype("category")

In [6]:
# dataset and dataloaders

# config
max_prediction_length = 6
max_encoder_length = 24
training_cutoff = data["time_idx"].max() - max_prediction_length

# dataset
training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="volume",
    group_ids=["agency", "sku"],
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["agency", "sku"],
    static_reals=["avg_population_2017", "avg_yearly_household_income_2017"],
    time_varying_known_categoricals=["special_days", "month"],
    variable_groups={"special_days": special_days},
    time_varying_known_reals=["time_idx", "price_regular", "discount_in_percent"],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=[
        "volume",
        "log_volume",
        "industry_volume",
        "soda_volume",
        "avg_max_temp",
        "avg_volume_by_agency",
        "avg_volume_by_sku",
    ],
    target_normalizer=GroupNormalizer(
        groups=["agency", "sku"], transformation="softplus"
    ),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)
validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

# dataloader
batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

In [7]:
columns = [
    "date", "time_idx", "month", 
    "agency", "sku", 
    "volume", "log_volume", "avg_volume_by_sku", "avg_volume_by_agency"
] + special_days
data[columns].head()

,date,time_idx,month,agency,sku,volume,log_volume,avg_volume_by_sku,avg_volume_by_agency,easter_day,...,new_year,christmas,labor_day,independence_day,revolution_day_memorial,regional_games,fifa_u_17_world_cup,football_gold_cup,beer_capital,music_fest
0,2013-01-01,0,1,Agency_22,SKU_01,52.2720,3.956461,2613.377501,103.805460,-,...,new_year,-,-,-,-,-,-,-,-,-
238,2013-01-01,0,1,Agency_37,SKU_04,0.0000,-18.420681,1361.511918,0.549900,-,...,new_year,-,-,-,-,-,-,-,-,-
237,2013-01-01,0,1,Agency_59,SKU_03,812.9214,6.700634,1225.306376,2041.909586,-,...,new_year,-,-,-,-,-,-,-,-,-
236,2013-01-01,0,1,Agency_11,SKU_01,316.4400,5.757134,2613.377501,125.690220,-,...,new_year,-,-,-,-,-,-,-,-,-
235,2013-01-01,0,1,Agency_05,SKU_05,420.9093,6.042417,1179.728165,1638.463500,-,...,new_year,-,-,-,-,-,-,-,-,-


In [8]:
# baseline model
baseline_predictions = Baseline().predict(val_dataloader, return_y=True)
MAE()(baseline_predictions.output, baseline_predictions.y)

2023-04-21 02:27:34.734748: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


tensor(293.0088)